In [ ]:
!sudo apt install tesseract-ocr

!pip install pytesseract

In [ ]:
import numpy as np
import cv2
import pandas as pd

In [ ]:
import imageio
import matplotlib.pyplot as plt
import numpy as np
from sklearn.cluster import KMeans
import cv2
from skimage.color import rgb2lab, deltaE_cie76
from collections import Counter
import os 


In [ ]:
import urllib.request
from urllib.error import URLError, HTTPError


In [ ]:
#you need to load data to colab for this to be available, then copy path
#urls expired need to be updated every 5 days
#upload data before running, data is deleted after closing the session and needs to be reloaded
db = pd.read_csv(r'')

In [ ]:
db.head(10)

## FUNCTIONS: 

In [ ]:
def url_to_image(url):
	# download the image, convert it to a NumPy array, and then read
	# it into OpenCV format
  resp = urllib.request.urlopen(url)
  image = np.asarray(bytearray(resp.read()), dtype="uint8")
  image = cv2.imdecode(image, cv2.IMREAD_COLOR)
	# return the image
  return image

In [ ]:
def RGB_HEX(color):
     return "#{:02x}{:02x}{:02x}".format(int(color[0]), int(color[1]), int(color[2]))

In [ ]:
def get_colors_as_list(image, number_of_colors):
     reshaped_image = cv2.resize(image, (600, 400))
     reshaped_image = reshaped_image.reshape(reshaped_image.shape[0]*reshaped_image.shape[1], 3)
     clf = KMeans(n_clusters = number_of_colors)
     labels = clf.fit_predict(reshaped_image)
     counts = Counter(labels)
     counts = dict(sorted(counts.items()))
     center_colors = clf.cluster_centers_
     ordered_colors = [center_colors[i] for i in counts.keys()]
     hex_colors = [RGB_HEX(ordered_colors[i]) for i in counts.keys()]
     rgb_colors = [ordered_colors[i] for i in counts.keys()]
     return hex_colors
#returns number of colors names based on hex

 


In [ ]:
def get_colors_as_count(image, number_of_colors):
     reshaped_image = cv2.resize(image, (600, 400))
     reshaped_image = reshaped_image.reshape(reshaped_image.shape[0]*reshaped_image.shape[1], 3)
     clf = KMeans(n_clusters = number_of_colors)
     labels = clf.fit_predict(reshaped_image)
     counts = Counter(labels)
     counts = dict(sorted(counts.items()))
     return counts
#returns how many pixels are estimated from each color

In [ ]:
db['Country'].unique() #remove from list countries with not latin abecedary

## Extracting images from URL

In [ ]:
from google.colab.patches import cv2_imshow #patch to fix crashes only for google colab

In [ ]:
#filtering by countries with roman letters

countries = ['Brazil', 'Mexico','Portugal',
       'United Kingdom', 'France', 'Canada', 'Spain', 'Poland',
       'United States', 'Italy', 'Ecuador', 'Australia', 'Netherlands',
       'Germany']
db = db[db['Country'].isin(countries)]

In [ ]:
db = db.reset_index(drop=True)

In [ ]:
db

In [ ]:
urls =db['Ad creative image URL'].tolist()
#urls expire so the next chapters of the code wont run if the links are expired, 
#click and validate before running

In [ ]:
print(urls)

In [ ]:
from urllib.error import URLError, HTTPError

In [ ]:
images = []
for entry in urls:
    try:
      resp = urllib.request.urlopen(entry)
    except HTTPError:
      print("url expired trying next one")
    else:
      image=url_to_image(entry)
      images.append(image)



In [ ]:
len(images)

In [ ]:
#image are read as a 'numpy.ndarray' type

## Extracting text from images

In [ ]:
# OCR libraries
import pytesseract

import shutil

import os

import random


In [ ]:
# read image (add index to select how many from the list)
img = images



In [ ]:
texts = []

for i in img:
  text = pytesseract.image_to_string(i)
  texts.append(text)
  

In [ ]:
len(texts)

## Extracting main colors from images

In [ ]:
#function to get colors from image based on kmeans and then plot them


In [ ]:
#list of top 5 main colors (this is meant to be used as dictionary key)
colors_list = []
for image in img:
  i = get_colors_as_list(image, 3)
  colors_list.append(i)
  




In [ ]:
color_counts_list = []
for image in img:
  i = get_colors_as_count(image, 3)
  color_counts_list.append(i) #count pixels with color to get proportions

In [ ]:
#adding from 2 functions color tags and pixel count
yeah = [dict(zip(colors_list[i], color_counts_list[i].values())) #zip merges 2 elements list like one on one
  for i in range(len(colors_list))]

In [ ]:
len(yeah)

In [ ]:
#creating df
colorsdf = pd.DataFrame.from_dict(yeah)

In [ ]:
colorsdf.shape

In [ ]:
colorsdf.to_csv(r'colordf.csv')

In [ ]:
listOfWidths = []
for image in img:
    width = image.shape[1]
    listOfWidths.append(width)

In [ ]:
len(listOfWidths)

In [ ]:
listOfHeights = []
for image in img:
    height = image.shape[0]
    listOfHeights.append(height)

In [ ]:
len(listOfHeights)

In [ ]:
heights = pd.DataFrame(listOfHeights, columns = ['height'])
widths = pd.DataFrame(listOfWidths, columns = ['width'])

In [ ]:
db['image_adHeight'] = heights['height']
db['image_adWidth'] = widths['width']

## Creating df

In [ ]:
#merging colors data to dataset

In [ ]:
db

In [ ]:
db_vf = pd.concat([db, colorsdf], axis=1) 

In [ ]:
db_vf.shape,db.shape,colorsdf.shape

In [ ]:
db_vf = db_vf.fillna(0)

In [ ]:
#creating df from text extracted from images
texts_df = pd.DataFrame(texts, columns = ['postImageText'])

In [ ]:
db_vf['postImageText'] = texts_df['postImageText']

In [ ]:
#saving
texts_df.to_csv(r'text_analysis.csv')

In [ ]:
#cleaning unwanted patterns use regex to identify them first
db_vf['postImageText'] = db_vf['postImageText'].str.replace('n\n',' ')

In [ ]:
db_vf['postImageText'] = db_vf['postImageText'].str.replace('\n','')

In [ ]:
db_vf['postImageText_len'] = texts_df['postImageText'].str.len()

In [ ]:
db_vf.shape

In [ ]:
#saving database
db_vf.to_csv(r'graphicdata_ProgrammaticCreativeAnalysis - extracted04-21-22.csv')

In [ ]:
db_vf

In [ ]:
#download to PC
from google.colab import files
files.download(r'graphicdata_ProgrammaticCreativeAnalysis - extracted04-21-22.csv')